# Settings

In [ ]:
is_plot = True
is_export = True

landmarks_path = '/Users/knpob/Territory/2-Kolmo/4-Dataset/20230715-DynaBreastLite/landmark/landmark.pkl'
meshes_path = '/Users/knpob/Territory/2-Kolmo/4-Dataset/20230715-DynaBreastLite/mesh/'

start=0
stride = 1
end=120

export_folder = "../output/dataset/"

# Data loading

In [ ]:
from mesh4d import obj3d

mesh_ls, texture_ls = obj3d.load_mesh_series(
    folder=meshes_path,
    start=start,
    stride=stride,
    end=end,
)

In [ ]:
from mesh4d import utils

landmarks = utils.load_pkl_object(landmarks_path)

In [ ]:
from mesh4d.analyse.crave import clip_with_contour

contour = landmarks.extract(('marker 0', 'marker 2', 'marker 3', 'marker 14', 'marker 15', 'marker 17'))
mesh_clip_ls = clip_with_contour(mesh_ls, start_time=0, fps=120/stride, contour=contour, clip_bound='xy', margin=30)

In [ ]:
body_ls = obj3d.init_obj_series(
    mesh_ls,
    obj_type=obj3d.Obj3d_Deform
    )

In [ ]:
breast_ls = obj3d.init_obj_series(
    mesh_clip_ls, 
    obj_type=obj3d.Obj3d_Deform
    )

In [ ]:
from mesh4d import obj4d

body4d = obj4d.Obj4d_Deform(
    fps=120 / stride,
    enable_rigid=False,
    enable_nonrigid=False,
)
body4d.add_obj(*body_ls)
body4d.load_markerset('landmarks', landmarks)

In [ ]:
from mesh4d import obj4d

breast4d = obj4d.Obj4d_Deform(
    fps=120 / stride,
    enable_rigid=False,
    enable_nonrigid=False,
)
breast4d.add_obj(*breast_ls)
breast4d.load_markerset('landmarks', landmarks)

# Data visualisation

## 4D body

In [ ]:
if is_export:
    body4d.animate(output_folder=export_folder, filename='body4d', elements='m')

In [ ]:
if is_export:
    body4d.animate(output_folder=export_folder, filename='body4d_kps')

In [ ]:
if is_plot:
    body4d.show(elements='m', window_size=[2000, 500], zoom_rate=5, skip=round(len(body_ls) / 10), m_props={'opacity': 0.5}, is_save=is_export, export_folder=export_folder, export_name='body4d_stack')

In [ ]:
if is_plot:
    body4d.show(elements='mk', window_size=[2000, 500], zoom_rate=5, skip=round(len(body_ls) / 10), m_props={'opacity': 0.5}, is_save=is_export, export_folder=export_folder, export_name='body4d_kps_stack')

## 4D breast

In [ ]:
if is_export:
    breast4d.animate(output_folder=export_folder, filename='breast4d', elements='m')

In [ ]:
if is_export:
    breast4d.animate(output_folder=export_folder, filename='breast4d_kps')

In [ ]:
import os
import mesh4d

if is_plot:
    import pyvista as pv

    stack_dist = 1000
    zoom_rate = 5
    window_size = [2000, 500]
    
    scene = pv.Plotter()
    plot_num = len(body4d.obj_ls)
    skip = round(len(body_ls) / 10)

    for idx in range(0, plot_num, skip):
        body = body4d.obj_ls[idx]
        breast = breast4d.obj_ls[idx]
        width = body.get_width()

        body.add_mesh_to_scene(scene, location=[0, 0, idx * stack_dist / skip], opacity=0.1)
        breast.add_mesh_to_scene(scene, location=[0, 0, idx * stack_dist / skip])
        breast.add_kps_to_scene(scene, location=[0, 0, idx * stack_dist / skip], radius=0.02*width)
        
    scene.camera_position = 'zy'
    scene.camera.azimuth = 45
    scene.camera.zoom(zoom_rate)
    scene.window_size = window_size
    scene.enable_parallel_projection()
    scene.show(interactive_update=True)

    if is_export:
        export_path = os.path.join(export_folder, 'breast_crop_stack.png')
        scene.update()
        scene.screenshot(export_path)
        
        if mesh4d.output_msg:
            print("export image: {}".format(export_path))



In [ ]:
if is_plot:
    scene = pv.Plotter()
    landmarks.add_to_scene(scene, trace_width=5)
    breast_ls[0].add_mesh_to_scene(scene, opacity=0.1)
    scene.camera_position = 'xy'

    if is_export:
        export_path = os.path.join(export_folder, 'stats', 'landmark_trace.png')
        scene.show(screenshot=export_path)

        if mesh4d.output_msg:
            print("export image: {}".format(export_path))
    else:
        scene.show()

# Statistics

## Collect data

In [ ]:
landmarks_full = landmarks.reslice(120)
frame_num = len(landmarks_full.markers['marker 0'].coord.T)
frame_num

In [ ]:
import pandas as pd

# init data frame
df = pd.DataFrame(columns=['Marker', 'Axis'] + list(n for n in range(frame_num)))

row = 0

for name, marker in landmarks_full.markers.items():
    for axis, sym in ((0, 'x (cm)'), (1, 'y (cm)'), (2, 'z (cm)')):
        row_data = [name, sym, *marker.coord[axis] / 10]
        df.loc[row] = row_data
        row = row + 1

df.head()

In [ ]:
df_styler = (df.set_index(["Marker", "Axis"]).T.describe().T.loc[:, "mean":]).style
df_styler.format(
    precision=2, 
    na_rep='--',
    )
df_styler

In [ ]:
print(df_styler.to_latex(
    position='h',
    position_float='centering',
    hrules=True,
))

## Overall trajectories

In [ ]:
import os
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 600

axis_ls = ('x', 'y', 'z')

In [ ]:
if is_plot:
    data = []
    labels = []

    for axis in axis_ls:
        data.append(df[df["Axis"]==axis].loc[:, 0:].to_numpy().reshape((-1,)))
        labels.append(f'{axis}-axis')

    bplot = plt.boxplot(
        x=data, labels=labels,
        patch_artist=True, showmeans=True, meanline=True, sym='',
        medianprops=dict(color='black'), meanprops=dict(color='black'),
        )

    for patch, color in zip(bplot['boxes'], ('goldenrod', 'teal', 'lightcoral')):
        patch.set_facecolor(color)

    plt.ylabel('Coordinates (cm)')
    plt.legend(
        bplot['boxes'],
        labels,
        )

    if is_export:
        plt.savefig(os.path.join(export_folder, 'stats', 'cooridates_distribute.png'))

In [ ]:
if is_plot:
    num = 0

    for axis in axis_ls:
        num = num + 1
        ax = plt.subplot(310 + num)
        df[df["Axis"]==axis].set_index("Marker").T.loc[0:].plot(ax=ax, legend=False)

        plt.title(f'{axis}-axis', fontsize=7)

        if num == round(len(axis_ls)/2):
            plt.ylabel('Coordinates (cm)')

        if num < len(axis_ls):
            plt.tick_params('x', labelbottom=False)
        else:
            plt.xlabel('Frame ID')

    if is_export:
        plt.savefig(os.path.join(export_folder, 'stats', f'cooridates_time.png'))

    plt.show()

## Trajectories length

In [ ]:
from mesh4d.analyse import measure

trace_dict, starts, traces = measure.markerset_trace_length(landmarks_full)

In [ ]:
import numpy as np
print(f'longest: marker {np.argmax(traces)}')
print(f'shortest: marker {np.argmin(traces)}')
print(f'ratio: {np.max(traces) / np.min(traces) * 100:.1f}%')

In [ ]:
if is_plot:
    for name, marker in trace_dict.items():
        plt.bar(name.split(' ')[-1], marker['trace'] / 10, alpha=0.8)

    plt.xlabel('Marker ID')
    plt.ylabel('Trajectory Length (cm)')

    if is_export:
        plt.savefig(os.path.join(export_folder, 'stats', f'trace_compare.png'))

    plt.show()

In [ ]:
if is_plot:
    for name, marker in trace_dict.items():
        acc_length = [
            marker['dist'][:n].sum() / 10
            for n in range(len(marker['dist']))
            ]
        plt.plot(acc_length, label=name)

    plt.xlabel('Frame ID')
    plt.ylabel('Accumulated Trajectory Length (cm)')

    if is_export:
        plt.savefig(os.path.join(export_folder, 'stats', f'trace_time.png'))

    plt.show()

In [ ]:
if is_plot:
    from mesh4d.analyse import visual

    visual.show_mesh_value_mask(
        breast_ls[1].mesh, starts, traces,
        k_nbr=20,
        is_save=is_export, export_folder=os.path.join(export_folder, 'stats'), export_name='breast_disp',
        show_edges=True, scalar_bar_args={'title': "tragcctory lenght (mm)"})